<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/Latent_Dirichlet_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#gensim
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [3]:
dataset=fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents=dataset.data
print('샘플의 수 : ', len(documents))

샘플의 수 :  11314


In [4]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [5]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [8]:
news_df=pd.DataFrame({'document':documents})
news_df.head()

,document
0,Well i'm not sure about the story nad it did s...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,Although I realize that principle is not one o...
3,Notwithstanding all the legitimate fuss about ...
4,"Well, I will have to change the scoring on my ..."


In [10]:
#특수 문자 제거
news_df['clean_doc']=news_df['document'].str.replace("[^a-zA-Z]", " ")
#길이가 3이하인 단어 제거
news_df['clean_doc']=news_df['clean_doc'].apply(lambda x:" ".join([w for w in x.split()if len(w)>3]))
#전체 단어 소문자화
news_df['clean_doc']=news_df['clean_doc'].apply(lambda x:x.lower())
news_df.head()

<ipython-input-10-c49d63b8af8f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc']=news_df['document'].str.replace("[^a-zA-Z]", " ")


,document,clean_doc
0,Well i'm not sure about the story nad it did s...,well sure about story seem biased what disagre...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re...",yeah expect people read actually accept hard a...
2,Although I realize that principle is not one o...,although realize that principle your strongest...
3,Notwithstanding all the legitimate fuss about ...,notwithstanding legitimate fuss about this pro...
4,"Well, I will have to change the scoring on my ...",well will have change scoring playoff pool unf...


In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
#NLTK로부터 불용어를 받아온다.
stop_words=stopwords.words('english')
tokenized_doc=news_df['clean_doc'].apply(lambda x:x.split())
tokenized_doc=tokenized_doc.apply(lambda x: [w for w in x if w not in stop_words])

In [18]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [19]:
from gensim import corpora
dictionary=corpora.Dictionary(tokenized_doc)#corpora의 Dictionary()를 활용해 손쉽게 word_id(정수 인코딩), word_frequency를 구할 수 있다.
corpus=[dictionary.doc2bow(text) for text in tokenized_doc]#tokenized_doc의 문장마다 bag of words(순서x, 빈도수만)로 변환하여 corpus에 저장한다.
print(corpus[1])

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [20]:
print(dictionary[56])

actually


In [21]:
len(dictionary)

64281

In [22]:
#LDA모델 훈련
import gensim
NUM_TOPICS=20 #k=20(토픽의 개수)
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15) #알고리즘은 15회 동작
topics=ldamodel.print_topics(num_words=4) #상위 4개까지만 토픽 출력
for topic in topics:
  print(topic)

(0, '0.014*"condition" + 0.011*"excellent" + 0.011*"sale" + 0.010*"asking"')
(1, '0.014*"points" + 0.014*"kent" + 0.012*"plane" + 0.011*"ticket"')
(2, '0.021*"armenian" + 0.017*"turkish" + 0.016*"armenians" + 0.011*"turkey"')
(3, '0.015*"objective" + 0.015*"moral" + 0.014*"morality" + 0.012*"absolute"')
(4, '0.016*"chip" + 0.015*"encryption" + 0.013*"keys" + 0.012*"clipper"')
(5, '0.012*"president" + 0.009*"health" + 0.006*"medical" + 0.006*"april"')
(6, '0.021*"entries" + 0.011*"earth" + 0.010*"moon" + 0.008*"winners"')
(7, '0.023*"game" + 0.020*"team" + 0.015*"games" + 0.014*"play"')
(8, '0.012*"people" + 0.010*"government" + 0.009*"israel" + 0.008*"state"')
(9, '0.014*"would" + 0.013*"like" + 0.011*"good" + 0.010*"time"')
(10, '0.012*"drive" + 0.011*"thanks" + 0.010*"system" + 0.009*"card"')
(11, '0.025*"radar" + 0.017*"detector" + 0.015*"easter" + 0.011*"forsale"')
(12, '0.010*"canada" + 0.009*"south" + 0.009*"north" + 0.007*"smokeless"')
(13, '0.017*"ground" + 0.014*"wire" + 0.011

In [26]:
#doc별 토픽 분포 보기
for i, topic_list in enumerate(ldamodel[corpus]):
  if i==5:
    break
  print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(2, 0.13670044), (6, 0.034077767), (8, 0.21771334), (9, 0.32524478), (12, 0.037763633), (17, 0.23700102)]
1 번째 문서의 topic 비율은 [(5, 0.028599767), (9, 0.2895932), (10, 0.05525419), (17, 0.60545963)]
2 번째 문서의 topic 비율은 [(8, 0.32811418), (9, 0.1086233), (17, 0.54952335)]
3 번째 문서의 topic 비율은 [(0, 0.018820604), (4, 0.3406559), (9, 0.41902396), (15, 0.057949603), (17, 0.12840773), (19, 0.023998864)]
4 번째 문서의 topic 비율은 [(4, 0.30224118), (7, 0.26408172), (9, 0.3333085), (16, 0.07072709)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
def make_topictable_per_doc(ldamodel, corpus):
  topic_table=pd.DataFrame()#저장할 데이터프레임

  #몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
  for i, topic_list in enumerate(ldamodel[corpus]):
    doc=topic_list[0] if ldamodel.per_word_topics else topic_list
    doc=sorted(doc, key=lambda x:(x[1]), reverse=True) #문서의 비중에 따라 내림차순 정렬

    for j, (topic_num, prop_topic) in enumerate(doc): #몇 번 토픽인지, 비중을 나눠서 저장
      if j==0:
        topic_table=topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]),ignore_index=True)
        #가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
      else:
        break
  return topic_table


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
topictable=make_topictable_per_doc(ldamodel, corpus)
topictable=topictable.reset_index() #문서 번호를 의미하는 인덱스 열 하나 추가
topictable.columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-27-8a64376a2ea3>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table=topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]),ignore_index=True)
<ipython-input-27-8a64376a2ea3>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table=topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]),ignore_index=True)
<ipython-input-27-8a64376a2ea3>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table=topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]),ignore_index=True)
<ipython-input-27-8a64376a2ea3>:11: FutureWarning: The frame.append method is deprecated and will be removed from p

In [30]:
topictable[:10]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,9,0.3252,"[(2, 0.13667098), (6, 0.03407746), (8, 0.21777..."
1,1,17,0.6056,"[(5, 0.028599765), (9, 0.2894069), (10, 0.0553..."
2,2,17,0.5496,"[(8, 0.32810977), (9, 0.10853041), (17, 0.5496..."
3,3,9,0.4191,"[(0, 0.018820055), (4, 0.34066865), (9, 0.4190..."
4,4,9,0.3333,"[(4, 0.30224138), (7, 0.26408178), (9, 0.33330..."
5,5,17,0.4971,"[(0, 0.20949396), (9, 0.12093773), (15, 0.1360..."
6,6,10,0.4746,"[(1, 0.020432746), (3, 0.03881114), (4, 0.0660..."
7,7,9,0.4220,"[(8, 0.16378431), (9, 0.42200324), (17, 0.2290..."
8,8,14,0.2592,"[(5, 0.15710531), (8, 0.1422992), (9, 0.194013..."
9,9,9,0.7089,"[(4, 0.11365494), (5, 0.015440982), (9, 0.7089..."
